In [46]:
import sys
sys.path.insert(1, '../')
from utils.utils_NMT import get_ckpt_folder
import os
from utils.utils_analyze_plots import get_ESD
from utils.utils_ww_results import get_all_layers
from utils.data_utils import get_data_loaders
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import pickle
from models.definitions.transformer_model import Transformer
from utils.constants import *
import torch
import weightwatcher as ww
import numpy as np
import powerlaw

In [47]:
print(ww.__file__)
print(powerlaw.__file__)

/home/ubuntu/test_ww/WeightWatcher2/WeightWatcher/weightwatcher/__init__.py
/home/ubuntu/test_ww/powerlaw/powerlaw.py


In [48]:
mypath = '../../checkpoint/NMT_epochs/IWSLT/w64/normal/metrics/epoch_0/'
all_layers = get_all_layers(mypath)

[2, 4, 21, 22, 23, 24, 28, 29, 42, 43, 44, 45, 49, 50, 63, 64, 65, 66, 70, 71, 84, 85, 86, 87, 91, 92, 105, 106, 107, 108, 112, 113, 126, 127, 128, 129, 133, 134, 153, 154, 155, 156, 161, 162, 163, 164, 168, 169, 185, 186, 187, 188, 193, 194, 195, 196, 200, 201, 217, 218, 219, 220, 225, 226, 227, 228, 232, 233, 249, 250, 251, 252, 257, 258, 259, 260, 264, 265, 281, 282, 283, 284, 289, 290, 291, 292, 296, 297, 313, 314, 315, 316, 321, 322, 323, 324, 328, 329, 334]


In [49]:
"""
['layer_id', 'name', 'D', 'M', 'N', 'W_scale', 'alpha', 'alpha_weighted',
       'best_fit', 'bulk_max', 'bulk_min', 'entropy', 'has_esd', 'lambda_max',
       'layer_type', 'log_alpha_norm', 'log_norm', 'log_spectral_norm',
       'matrix_rank', 'max_rand_eval', 'mp_softrank', 'norm', 'num_evals',
       'num_pl_spikes', 'num_spikes', 'rand_W_scale', 'rand_bulk_max',
       'rand_bulk_min', 'rand_mp_softrank', 'rand_num_spikes', 'rand_sigma_mp',
       'rank_loss', 'rf', 'sigma', 'sigma_mp', 'spectral_norm', 'stable_rank',
       'sv_max', 'warning', 'xmax', 'xmin']
"""

"\n['layer_id', 'name', 'D', 'M', 'N', 'W_scale', 'alpha', 'alpha_weighted',\n       'best_fit', 'bulk_max', 'bulk_min', 'entropy', 'has_esd', 'lambda_max',\n       'layer_type', 'log_alpha_norm', 'log_norm', 'log_spectral_norm',\n       'matrix_rank', 'max_rand_eval', 'mp_softrank', 'norm', 'num_evals',\n       'num_pl_spikes', 'num_spikes', 'rand_W_scale', 'rand_bulk_max',\n       'rand_bulk_min', 'rand_mp_softrank', 'rand_num_spikes', 'rand_sigma_mp',\n       'rank_loss', 'rf', 'sigma', 'sigma_mp', 'spectral_norm', 'stable_rank',\n       'sv_max', 'warning', 'xmax', 'xmin']\n"

## Try if we can load the model itself

In [50]:
blue, yellow, red = '#91bfdb', '#ffffbf', '#fc8d59'

In [55]:
def plot_loghist(x, bins, xmin):
    hist, bins = np.histogram(x, bins=bins)
    logbins = np.logspace(np.log10(bins[0]),np.log10(bins[-1]),len(bins))
    plt.hist(x, bins=logbins, density=True, color=blue)

    if xmin:
        plt.axvline(xmin, color='r', label=r'$x_{min}$')

    plt.xscale('log')

In [52]:
dataset="WMT"
num_samples=1280000

_, _, src_field_processor, trg_field_processor = get_data_loaders(
        '../data',
        'G2E',
        dataset,
        1500,
        torch.device("cpu"),
        subsampling=True,
        num_samples=num_samples)

Use only 1280000 samples.
train dataset (WMT-14) has 37491570 tokens in the source language (German) corpus.
train dataset (WMT-14) has 36958381 tokens in the target language (English) corpus.
val dataset (WMT-14) has 81809 tokens in the source language (German) corpus.
val dataset (WMT-14) has 76782 tokens in the target language (English) corpus.
Time it took to prepare the data: 87.874141 seconds.


In [56]:
def plot_result_new(experiment_type, width, training_type, epoch, layer_id, num_layers, distribution, id, fig_title):
    
    # Load checkpoint
    ckpt_folder = get_ckpt_folder(experiment_type=experiment_type, dataset=dataset, num_samples=num_samples, 
                                  width=width, training_type=training_type, directory_depth=1, 
                                  lr_factor=None, folder_suffix='', depth=None)
    #print(ckpt_folder)

    ckpt = os.path.join(ckpt_folder, f'net_epoch_{epoch}.ckpt')
    #pad_token_id = src_field_processor.vocab.stoi[PAD_TOKEN]  # pad token id is the same for target as well
    src_vocab_size = len(src_field_processor.vocab)
    trg_vocab_size = len(trg_field_processor.vocab)

    baseline_transformer = Transformer(
            model_dimension=width,
            src_vocab_size=src_vocab_size,
            trg_vocab_size=trg_vocab_size,
            number_of_heads=BASELINE_MODEL_NUMBER_OF_HEADS,
            number_of_layers=num_layers,
            dropout_probability=BASELINE_MODEL_DROPOUT_PROB
    )

    ckpt = torch.load(ckpt, map_location='cpu')
    baseline_transformer.load_state_dict(ckpt["state_dict"])
    
    # Get evals
    watcher = ww.WeightWatcher(model=baseline_transformer)
    #watcher.analyze(distribution=distribution, layers=[layer_id], plot=True, mp_fit=True, randomize=True)    
    evals = watcher.get_ESD(layer=layer_id)
    
    # Fit evals
    EVALS_THRESH = 0.00001
    if id !=0:
        # Fit truncated powerlaw to this distribution using fix finger
        nz_evals = evals[evals > EVALS_THRESH]
        xmax = np.max(evals)
        num_bins = 100  # np.min([100, len(nz_evals)])
        h = np.histogram(np.log10(nz_evals), bins=num_bins)
        ih = np.argmax(h[0])
        xmin2 = 10 ** h[1][ih]
        xmin_range = (0.95 * xmin2, 1.05 * xmin2)

        fit = powerlaw.Fit(evals, xmin=xmin_range, xmax=xmax, verbose=False, xmin_distribution=distribution)
        
    else:
        # Fit ordinary powerlaw
        xmax = np.max(evals)
        nz_evals = evals[evals > EVALS_THRESH]
        fit = powerlaw.Fit(nz_evals, xmax=xmax, verbose=False)

    # Plot evals
    xmin = fit.xmin
    xmax = fit.xmax

    fig2 = fit.plot_pdf(color='b', linewidth=0) # invisbile
    #plot_loghist(evals[evals>(xmin/100)], bins=100, xmin=xmin)
    plot_loghist(evals, bins=100, xmin=xmin)
    fig2 = fit.plot_pdf(color='r', linewidth=2)
    if distribution == "power_law":
        fit.power_law.plot_pdf(color='r', linestyle='--', ax=fig2)
    elif distribution == "truncated_power_law":
        fit.truncated_power_law.plot_pdf(color='r', linestyle='--', ax=fig2)
    elif distribution == "exponential":
        fit.exponential.plot_pdf(color='r', linestyle='--', ax=fig2)
    elif distribution == "lognormal":
        fit.lognormal.plot_pdf(color='r', linestyle='--', ax=fig2)

    fig2.set_xlabel("Singular values", fontsize=14)
    fig2.set_ylabel("ESD", fontsize=14)
    fig2.figure.set_size_inches(3.8, 3.2)
    
    fig2.set_xlim([0.01, 100])
    fig2.set_ylim([0.0001, 10])
    plt.legend()

    alpha = "{:.2f}".format(fit.alpha)
    KS = "{:.2f}".format(fit.D)
    exp = "{:.2f}".format(fit.exponent)
    sigma = "{:.2f}".format(fit.sigma)
    
    if id==0:
        title = f"alpha={alpha}, ks_distance={KS}"
    elif id==1:
        title = f"E-TPL alpha={alpha}, ks_distance={KS}" + f"\n E-TPL exponent={exp}"
    elif id==2:
        title = f"sigma={sigma}, ks_distance={KS}"
    elif id==3:
        title = f"exponent={exp}, ks_distance={KS}"        
            
    plt.title(title, fontsize=14)
    
    plt.tight_layout()
    plt.savefig(f"{fig_title}_{id}.pdf", dpi=300, format='pdf')
    plt.clf()

In [57]:
plt.rcParams['figure.dpi'] = 300
experiment_type="sample"
width=512
training_type="normal"
num_layers = 6

distributions = ["power_law", "truncated_power_law", "lognormal", "exponential"]

# Good PL fitting

In [58]:
epoch, layer_id = 6, 322
for id, distribution in zip(range(4), distributions):
    plot_result_new(experiment_type, width, training_type, epoch, 
                    layer_id, num_layers, distribution, id, fig_title='results/TPL_vs_PL_good')

INFO:weightwatcher:

python      version 3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
numpy       version 1.20.3
tensforflow version 2.6.0
keras       version 2.6.0
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_svd', 'fix_fingers': False, 'min_evals': 0, 'max_evals': None, 'mp_fit': False, 'layers': []}
INFO:weightwatcher:Saving all images to ww-img
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_s

INFO:weightwatcher:Getting ESD for layer 322 


INFO:weightwatcher:

python      version 3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
numpy       version 1.20.3
tensforflow version 2.6.0
keras       version 2.6.0
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_svd', 'fix_fingers': False, 'min_evals': 0, 'max_evals': None, 'mp_fit': False, 'layers': []}
INFO:weightwatcher:Saving all images to ww-img
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_s

INFO:weightwatcher:Getting ESD for layer 322 


INFO:weightwatcher:

python      version 3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
numpy       version 1.20.3
tensforflow version 2.6.0
keras       version 2.6.0
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_svd', 'fix_fingers': False, 'min_evals': 0, 'max_evals': None, 'mp_fit': False, 'layers': []}
INFO:weightwatcher:Saving all images to ww-img
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_s

INFO:weightwatcher:Getting ESD for layer 322 


INFO:weightwatcher:

python      version 3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
numpy       version 1.20.3
tensforflow version 2.6.0
keras       version 2.6.0
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_svd', 'fix_fingers': False, 'min_evals': 0, 'max_evals': None, 'mp_fit': False, 'layers': []}
INFO:weightwatcher:Saving all images to ww-img
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_s

INFO:weightwatcher:Getting ESD for layer 322 


<Figure size 1140x960 with 0 Axes>

# Mediocre PL fitting

In [59]:
epoch, layer_id = 15, 290
for id, distribution in zip(range(4), distributions):
    plot_result_new(experiment_type, width, training_type, epoch, 
                    layer_id, num_layers, distribution, id, fig_title='results/TPL_vs_PL_mediocre')

INFO:weightwatcher:

python      version 3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
numpy       version 1.20.3
tensforflow version 2.6.0
keras       version 2.6.0
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_svd', 'fix_fingers': False, 'min_evals': 0, 'max_evals': None, 'mp_fit': False, 'layers': []}
INFO:weightwatcher:Saving all images to ww-img
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_s

INFO:weightwatcher:Getting ESD for layer 290 


INFO:weightwatcher:

python      version 3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
numpy       version 1.20.3
tensforflow version 2.6.0
keras       version 2.6.0
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_svd', 'fix_fingers': False, 'min_evals': 0, 'max_evals': None, 'mp_fit': False, 'layers': []}
INFO:weightwatcher:Saving all images to ww-img
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_s

INFO:weightwatcher:Getting ESD for layer 290 


INFO:weightwatcher:

python      version 3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
numpy       version 1.20.3
tensforflow version 2.6.0
keras       version 2.6.0
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_svd', 'fix_fingers': False, 'min_evals': 0, 'max_evals': None, 'mp_fit': False, 'layers': []}
INFO:weightwatcher:Saving all images to ww-img
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_s

INFO:weightwatcher:Getting ESD for layer 290 


INFO:weightwatcher:

python      version 3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
numpy       version 1.20.3
tensforflow version 2.6.0
keras       version 2.6.0
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_svd', 'fix_fingers': False, 'min_evals': 0, 'max_evals': None, 'mp_fit': False, 'layers': []}
INFO:weightwatcher:Saving all images to ww-img
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_s

INFO:weightwatcher:Getting ESD for layer 290 


<Figure size 1140x960 with 0 Axes>

# Bad PL fitting

In [60]:
epoch, layer_id = 4, 284
for id, distribution in zip(range(4), distributions):
    plot_result_new(experiment_type, width, training_type, epoch, 
                    layer_id, num_layers, distribution, id, fig_title='results/TPL_vs_PL_bad')

INFO:weightwatcher:

python      version 3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
numpy       version 1.20.3
tensforflow version 2.6.0
keras       version 2.6.0
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_svd', 'fix_fingers': False, 'min_evals': 0, 'max_evals': None, 'mp_fit': False, 'layers': []}
INFO:weightwatcher:Saving all images to ww-img
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_s

INFO:weightwatcher:Getting ESD for layer 284 


INFO:weightwatcher:

python      version 3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
numpy       version 1.20.3
tensforflow version 2.6.0
keras       version 2.6.0
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_svd', 'fix_fingers': False, 'min_evals': 0, 'max_evals': None, 'mp_fit': False, 'layers': []}
INFO:weightwatcher:Saving all images to ww-img
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_s

INFO:weightwatcher:Getting ESD for layer 284 


INFO:weightwatcher:

python      version 3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
numpy       version 1.20.3
tensforflow version 2.6.0
keras       version 2.6.0
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_svd', 'fix_fingers': False, 'min_evals': 0, 'max_evals': None, 'mp_fit': False, 'layers': []}
INFO:weightwatcher:Saving all images to ww-img
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_s

INFO:weightwatcher:Getting ESD for layer 284 


INFO:weightwatcher:

python      version 3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
numpy       version 1.20.3
tensforflow version 2.6.0
keras       version 2.6.0
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_svd', 'fix_fingers': False, 'min_evals': 0, 'max_evals': None, 'mp_fit': False, 'layers': []}
INFO:weightwatcher:Saving all images to ww-img
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_s

INFO:weightwatcher:Getting ESD for layer 284 


<Figure size 1140x960 with 0 Axes>

# Extremely good fitting

In [61]:
dataset="WMT"
num_samples=640000

_, _, src_field_processor, trg_field_processor = get_data_loaders(
        '../data',
        'G2E',
        dataset,
        1500,
        torch.device("cpu"),
        subsampling=True,
        num_samples=num_samples)

Use only 640000 samples.
train dataset (WMT-14) has 18878961 tokens in the source language (German) corpus.
train dataset (WMT-14) has 18751301 tokens in the target language (English) corpus.
val dataset (WMT-14) has 81809 tokens in the source language (German) corpus.
val dataset (WMT-14) has 76782 tokens in the target language (English) corpus.
Time it took to prepare the data: 77.719848 seconds.


In [62]:
def plot_result_new(experiment_type, width, training_type, epoch, layer_id, num_layers, distribution, id, fig_title):
    
    # Load checkpoint
    ckpt_folder = get_ckpt_folder(experiment_type=experiment_type, dataset=dataset, num_samples=num_samples, 
                                  width=width, training_type=training_type, directory_depth=1, 
                                  lr_factor=None, folder_suffix='', depth=None)
    #print(ckpt_folder)

    ckpt = os.path.join(ckpt_folder, f'net_epoch_{epoch}.ckpt')
    #pad_token_id = src_field_processor.vocab.stoi[PAD_TOKEN]  # pad token id is the same for target as well
    src_vocab_size = len(src_field_processor.vocab)
    trg_vocab_size = len(trg_field_processor.vocab)

    baseline_transformer = Transformer(
            model_dimension=width,
            src_vocab_size=src_vocab_size,
            trg_vocab_size=trg_vocab_size,
            number_of_heads=BASELINE_MODEL_NUMBER_OF_HEADS,
            number_of_layers=num_layers,
            dropout_probability=BASELINE_MODEL_DROPOUT_PROB
    )

    ckpt = torch.load(ckpt, map_location='cpu')
    baseline_transformer.load_state_dict(ckpt["state_dict"])
    
    # Get evals
    watcher = ww.WeightWatcher(model=baseline_transformer)
    #watcher.analyze(distribution=distribution, layers=[layer_id], plot=True, mp_fit=True, randomize=True)    
    evals = watcher.get_ESD(layer=layer_id)
    
    # Fit evals
    EVALS_THRESH = 0.00001
    if id !=0:
        # Fit truncated powerlaw to this distribution using fix finger
        nz_evals = evals[evals > EVALS_THRESH]
        xmax = np.max(evals)
        num_bins = 100  # np.min([100, len(nz_evals)])
        h = np.histogram(np.log10(nz_evals), bins=num_bins)
        ih = np.argmax(h[0])
        xmin2 = 10 ** h[1][ih]
        xmin_range = (0.95 * xmin2, 1.05 * xmin2)

        fit = powerlaw.Fit(evals, xmin=xmin_range, xmax=xmax, verbose=False, xmin_distribution=distribution)
        
    else:
        # Fit ordinary powerlaw
        xmax = np.max(evals)
        nz_evals = evals[evals > EVALS_THRESH]
        fit = powerlaw.Fit(nz_evals, xmax=xmax, verbose=False)

    # Plot evals
    xmin = fit.xmin
    xmax = fit.xmax

    fig2 = fit.plot_pdf(color='b', linewidth=0) # invisbile
    #plot_loghist(evals[evals>(xmin/100)], bins=100, xmin=xmin)
    plot_loghist(evals, bins=100, xmin=xmin)
    fig2 = fit.plot_pdf(color='r', linewidth=2)
    if distribution == "power_law":
        fit.power_law.plot_pdf(color='r', linestyle='--', ax=fig2)
    elif distribution == "truncated_power_law":
        fit.truncated_power_law.plot_pdf(color='r', linestyle='--', ax=fig2)
    elif distribution == "exponential":
        fit.exponential.plot_pdf(color='r', linestyle='--', ax=fig2)
    elif distribution == "lognormal":
        fit.lognormal.plot_pdf(color='r', linestyle='--', ax=fig2)

    fig2.set_xlabel("Singular values", fontsize=14)
    fig2.set_ylabel("ESD", fontsize=14)
    fig2.figure.set_size_inches(3.8, 3.2)
    
    fig2.set_xlim([0.8, 100])
    fig2.set_ylim([0.0001, 10])
    plt.legend()

    alpha = "{:.2f}".format(fit.alpha)
    KS = "{:.2f}".format(fit.D)
    exp = "{:.2f}".format(fit.exponent)
    sigma = "{:.2f}".format(fit.sigma)
    
    if id==0:
        title = f"alpha={alpha}, ks_distance={KS}"
    elif id==1:
        title = f"E-TPL alpha={alpha}, ks_distance={KS}" + f"\n E-TPL exponent={exp}"
    elif id==2:
        title = f"sigma={sigma}, ks_distance={KS}"
    elif id==3:
        title = f"exponent={exp}, ks_distance={KS}"        
            
    plt.title(title, fontsize=14)
    
    plt.tight_layout()
    plt.savefig(f"{fig_title}_{id}.pdf", dpi=300, format='pdf')
    plt.clf()

In [63]:
epoch, layer_id = 2, 296
for id, distribution in zip(range(4), distributions):
    plot_result_new(experiment_type, width, training_type, epoch, 
                    layer_id, num_layers, distribution, id, fig_title='results/TPL_vs_PL_good')

INFO:weightwatcher:

python      version 3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
numpy       version 1.20.3
tensforflow version 2.6.0
keras       version 2.6.0
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_svd', 'fix_fingers': False, 'min_evals': 0, 'max_evals': None, 'mp_fit': False, 'layers': []}
INFO:weightwatcher:Saving all images to ww-img
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_s

INFO:weightwatcher:Getting ESD for layer 296 


INFO:weightwatcher:

python      version 3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
numpy       version 1.20.3
tensforflow version 2.6.0
keras       version 2.6.0
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_svd', 'fix_fingers': False, 'min_evals': 0, 'max_evals': None, 'mp_fit': False, 'layers': []}
INFO:weightwatcher:Saving all images to ww-img
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_s

INFO:weightwatcher:Getting ESD for layer 296 


INFO:weightwatcher:

python      version 3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
numpy       version 1.20.3
tensforflow version 2.6.0
keras       version 2.6.0
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_svd', 'fix_fingers': False, 'min_evals': 0, 'max_evals': None, 'mp_fit': False, 'layers': []}
INFO:weightwatcher:Saving all images to ww-img
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_s

INFO:weightwatcher:Getting ESD for layer 296 


INFO:weightwatcher:

python      version 3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
numpy       version 1.20.3
tensforflow version 2.6.0
keras       version 2.6.0
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_svd', 'fix_fingers': False, 'min_evals': 0, 'max_evals': None, 'mp_fit': False, 'layers': []}
INFO:weightwatcher:Saving all images to ww-img
INFO:weightwatcher:params {'glorot_fix': False, 'normalize': False, 'conv2d_norm': True, 'randomize': False, 'savedir': 'ww-img', 'savefig': 'ww-img', 'rescale': True, 'plot': False, 'deltaEs': False, 'intra': False, 'channels': None, 'conv2d_fft': False, 'ww2x': False, 'vectors': False, 'smooth': None, 'stacked': False, 'svd_method': 'full_s

INFO:weightwatcher:Getting ESD for layer 296 


<Figure size 1140x960 with 0 Axes>

# Old code

In [ ]:
def plot_result(epoch, layer_id, ckpt_folder, fig_title='fig'):
    
    layer_id_ww = all_layers.index(layer_id)
    for id, metric_folder_name in [(0, 'metrics'), (1, 'metrics_expcutoff')]:
        
        fig, ax = plt.subplots(1, 1, figsize=(3.5, 2.8))
        ESD_result = get_ESD(ckpt_folder, epoch=epoch, ESD_type=1,
                         layer_id=layer_id, metric_folder_name = metric_folder_name)
        WW_result = os.path.join(ckpt_folder, metric_folder_name, f'epoch_{epoch}', 'results.pkl')
        details = pickle.load(open(WW_result, 'rb'))['details']
        alpha = "{:.2f}".format(details['alpha'][layer_id_ww])
        KS = "{:.2f}".format(details['D'][layer_id_ww])
        title = f"Alpha={alpha}, KS_distance={KS}"
        if id==1:
            exp = "{:.2f}".format(details['exponent'][layer_id_ww])
            title += f"\n EXPONENT={exp}"

        img = mpimg.imread(ESD_result)
        #print(img.shape)
        #img_cropped = img[40:460, 40:580, :]
        img_cropped = img
        ax.imshow(img_cropped)
        ax.set_title(title)
        ax.axis('off')
        plt.tight_layout()
        plt.savefig(f"{fig_title}_{id}.pdf", dpi=300, format='pdf')

# Good PL fitting

In [ ]:
plt.rcParams['figure.dpi'] = 300
ckpt_folder = get_ckpt_folder(experiment_type="sample", dataset="WMT", num_samples=1280000, 
                              width=512, training_type="normal", directory_depth=1, 
                              lr_factor=None, folder_suffix='', depth=None)

#epoch, layer_id = 19, 323
epoch, layer_id = 4, 291
plot_result(epoch, layer_id, ckpt_folder, fig_title='results/TPL_vs_PL_good_old')

# Mediocre PL fitting

In [ ]:
plt.rcParams['figure.dpi'] = 300
ckpt_folder = get_ckpt_folder(experiment_type="sample", dataset="WMT", num_samples=1280000, 
                              width=512, training_type="normal", directory_depth=1, 
                              lr_factor=None, folder_suffix='', depth=None)

epoch, layer_id = 15, 290
plot_result(epoch, layer_id, ckpt_folder, fig_title='results/TPL_vs_PL_mediocre_old')

# Bad PL fitting

In [ ]:
plt.rcParams['figure.dpi'] = 300
ckpt_folder = get_ckpt_folder(experiment_type="sample", dataset="WMT", num_samples=1280000,
                              width=512, training_type="normal", directory_depth=1,
                              lr_factor=None, folder_suffix='', depth=None)

epoch, layer_id = 4, 284
plot_result(epoch, layer_id, ckpt_folder, fig_title='results/TPL_vs_PL_bad_old')